In [1]:
import pandas as pd

# Load the data
data_path = './data_and_materials/gamma-ray.csv'
gamma_ray_data = pd.read_csv(data_path)

# Display the first few rows of the data to understand its structure
gamma_ray_data.head()


,seconds,count
0,116.0,0.0
1,112.0,0.0
2,160.0,0.0
3,51.5,0.0
4,102.0,1.0


In [2]:
# Calculate total observation time and total number of events
total_time = gamma_ray_data['seconds'].sum()
total_events = gamma_ray_data['count'].sum()

# Estimate the average rate of gamma ray emissions per second
lambda_estimate = total_events / total_time

total_time, total_events, lambda_estimate

(15718.2, 61.0, 0.0038808514969907496)

In [3]:
# Calculate the MLE for lambda
lambda_mle = total_events / total_time

lambda_mle


0.0038808514969907496

In [4]:
from scipy.stats import chi2

# Determine the critical value for the chi-squared distribution with 99 degrees of freedom at the 95th percentile
critical_value = chi2.ppf(0.95, df=99)

critical_value

123.2252214533618

In [10]:
import numpy as np
from scipy.special import gammaln

# Define log-likelihood calculation functions
def log_likelihood_null(data, lambda_mle):
    return np.sum(data['count'] * np.log(lambda_mle * data['seconds'] + 1e-10) - 
                  lambda_mle * data['seconds'] - 
                  gammaln(data['count'] + 1))

def log_likelihood_alternative(data):
    return np.sum(data.apply(lambda row: row['count'] * np.log(row['count'] + 1e-10) - 
                                        row['count'] - 
                                        gammaln(row['count'] + 1), axis=1))

# Compute log-likelihoods
log_likelihood_null_value = log_likelihood_null(gamma_ray_data, lambda_mle)
log_likelihood_alternative_value = log_likelihood_alternative(gamma_ray_data)

# Compute test statistic
test_statistic_final = -2 * (log_likelihood_null_value - log_likelihood_alternative_value)

# Compute p-value
p_value_final = chi2.sf(test_statistic_final, df=99)

test_statistic_final, p_value_final


(104.39792131493309, 0.3357666244071094)